# Deep Learning
## Assignment 1 - Beat the market
### Abel de Wit & Malin Hjärtström


In [1]:
# Getting the data (commented for local use)
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
import tensorflow as tf
print(tf.__version__)

2.0.0


In [2]:
import pandas as pd
from keras.models import Sequential
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [3]:
info_data = pd.read_csv("data/info.txt", sep='\s+')
market_analysis = pd.read_csv("data/market_analysis.txt", sep='\s+')
market_segments = pd.read_csv("data/market_segments.txt", sep='\s+')
stock_prices = pd.read_csv("data/stock_prices.txt", sep='\s+')

# Do something with all our data so we can feed it to the NN
dataframe = info_data

# 'One hot encoding' the segments
dataframe["IT"] = dataframe['company'].apply(lambda x: 0 if x == 1 else 1)
dataframe["BIO"] = dataframe['company'].apply(lambda x: 1 if x == 1 else 0)
#dataframe["trend"] = market_analysis['trend']
dataframe["stock-price"] = stock_prices['stock-price']

# Setting the indexes as the date
dataframe.set_index(['year', 'day'], inplace=True)


# For now we dont use some data
# del dataframe['sentiment']
# del dataframe['m1']
# del dataframe['m2']
# del dataframe['m3']
# del dataframe['m4']

In [4]:
# Now we split into companies

company_0 = dataframe[dataframe['company'] == 0]
company_1 = dataframe[dataframe['company'] == 1]
company_2 = dataframe[dataframe['company'] == 2]

# Let's see how their stocks are doing

company_0.plot(y='stock-price').set_title('Company 0')
company_1.plot(y='stock-price').set_title('Company 1')
company_2.plot(y='stock-price').set_title('Company 2')

del company_0['company']
del company_0['quarter']
del company_1['company']
del company_1['quarter']
del company_2['company']
del company_2['quarter']

We want to predict wether the stock goes up or not, so we have to change the stock price values in such a way that it is binary.

`if stock-price-today - stock-price-yesterday > 0 then 1, else 0`

In [5]:
import numpy as np
company_0['stock-price-binary'] = np.where(company_0['stock-price'] > company_0['stock-price'].shift(), 1, 0)
del company_0['stock-price']
company_0

/Users/abel/anaconda3/envs/BeatTheMarket/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


expert1  expert2  sentiment   m1    m2   m3  m4  IT  BIO  \
year day                                                             
2017 3          0        0         10  6.3  1824 -1.0   0   1    0   
     4          0        1         10  5.1  6912 -0.9   0   1    0   
     5          0        1         10  6.6  8928  0.3   0   1    0   
     6          0        1         10  7.8  6924  0.0   0   1    0   
     9          0        1         10 -0.9  5635  0.9   0   1    0   
...           ...      ...        ...  ...   ...  ...  ..  ..  ...   
2019 175        1        0          4  7.8  4444 -0.9   0   1    0   
     176        0        0          5  6.8  5901 -0.7   0   1    0   
     177        0        0          4  8.1  1631  0.0   0   1    0   
     178        0        0          5  4.3   352 -0.9   0   1    0   
     179        0        1          5 -1.2  5765  0.1   0   1    0   

          stock-price-binary  
year day                      
2017 3                     0  
     4                     0  
     5                     0  
     6                     0  
     9                     0  
...                      ...  
2019 175                   0  
     176                   0  
     177                   0  
     178                   0  
     179                   1  

[626 rows x 10 columns]

# Company 1

So now we have the data in a nice table, split into seperate companies, we can do some machine learning!

In [6]:
# Scale data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaled_0 = scaler.fit_transform(company_0)
scaled_0 = pd.DataFrame(data=scaled_0, columns=company_0.columns)

X = scaled_0.loc[:, scaled_0.columns != 'stock-price-binary']
y = scaled_0['stock-price-binary']

print(X.shape)
print(y.shape)

# Now we split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

(626, 9)
(626,)


In [9]:
import datetime, os
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, LSTM
from keras import regularizers


# Create a model.
model = Sequential()
model.add(Dense(64, input_dim=9, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, input_dim=9, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# Train the model
log_dir = os.path.join(
    "logs",
    "fit",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir, histogram_freq=1)

model.fit(x=X_train, 
          y=y_train, 
          epochs=100, 
          validation_data=(X_test, y_test),
          callbacks=[tensorboard_callback],
          verbose=1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                640       
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 520       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 9         
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________
Train on 500 samples, validate on 126 samples
Epoch 1/100
500/500 [==============================] - 0s 483us/step - loss: 0.6633 - accuracy: 0.6200 - val_loss: 0.6195 - val_accuracy: 0.7143
Epoch 2/100
500/500 [==============================] - 0s 4

Epoch 51/100
500/500 [==============================] - 0s 42us/step - loss: 0.3078 - accuracy: 0.8660 - val_loss: 0.2433 - val_accuracy: 0.8968
Epoch 52/100
500/500 [==============================] - 0s 36us/step - loss: 0.2991 - accuracy: 0.8680 - val_loss: 0.2436 - val_accuracy: 0.8968
Epoch 53/100
500/500 [==============================] - 0s 37us/step - loss: 0.3121 - accuracy: 0.8680 - val_loss: 0.2435 - val_accuracy: 0.8968
Epoch 54/100
500/500 [==============================] - 0s 36us/step - loss: 0.2948 - accuracy: 0.8780 - val_loss: 0.2443 - val_accuracy: 0.8810
Epoch 55/100
500/500 [==============================] - 0s 36us/step - loss: 0.3063 - accuracy: 0.8740 - val_loss: 0.2436 - val_accuracy: 0.8968
Epoch 56/100
500/500 [==============================] - 0s 38us/step - loss: 0.2935 - accuracy: 0.8820 - val_loss: 0.2448 - val_accuracy: 0.8968
Epoch 57/100
500/500 [==============================] - 0s 39us/step - loss: 0.2988 - accuracy: 0.8740 - val_loss: 0.2437 - val_ac

In [10]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 9736), started 12:40:24 ago. (Use '!kill 9736' to kill it.)